In [1]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

# dimensions of our images.
img_width, img_height = 150, 150

top_model_weights_path = 'bottleneck_fc_model.h5'
train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16


def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('bottleneck_features_train.npy', 'w'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('bottleneck_features_validation.npy', 'w'),
            bottleneck_features_validation)


def train_top_model():
    train_data = np.load(open('bottleneck_features_train.npy'))
    train_labels = np.array(
        [0] * (nb_train_samples / 2) + [1] * (nb_train_samples / 2))

    validation_data = np.load(open('bottleneck_features_validation.npy'))
    validation_labels = np.array(
        [0] * (nb_validation_samples / 2) + [1] * (nb_validation_samples / 2))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)


save_bottlebeck_features()
train_top_model()

Using TensorFlow backend.


Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Train on 2000 samples, validate on 800 samples
Epoch 1/50
2000/2000 [==============================] - 3s 2ms/step - loss: 0.7311 - acc: 0.7550 - val_loss: 0.6776 - val_acc: 0.7150
Epoch 2/50
2000/2000 [==============================] - 3s 2ms/step - loss: 0.4218 - acc: 0.8200 - val_loss: 0.2818 - val_acc: 0.8712
Epoch 3/50
2000/2000 [==============================] - 3s 2ms/step - loss: 0.3212 - acc: 0.8750 - val_loss: 0.5680 - val_acc: 0.7837
Epoch 4/50
2000/2000 [==============================] - 3s 2ms/step - loss: 0.3134 - acc: 0.8760 - val_loss: 0.3246 - val_acc: 0.8650
Epoch 5/50
2000/2000 [==============================] - 3s 2ms/step - loss: 0.2504 - acc: 0.8975 - val_loss: 0.2849 - val_acc: 0.8938
Epoch 6/50
2000/2000 [==============================] - 3s 2ms/step - loss: 0.2480 - acc: 0.9135 - val_loss: 0.2776 - val_acc: 0.8912
Epoch 7/50
2000/2000 [==============================] - 3s 2ms/ste